# Reading blocks and putting it in one dataframe

In [1]:
import os, re
import pandas as pd
from ast import literal_eval

In [2]:
try:
    Exist=pd.read_csv('blockchain.csv')['hash'].to_list()
except:
    Exist=[]
files = [f for f in os.listdir('./raw blocks/')]

In [3]:
Exist=[ex+'.blk' for ex in Exist]

In [ ]:
files = [f for f in os.listdir('./raw blocks/')]
files=list(set(Exist)^set(files))
files=['./raw blocks/'+file for file in files if re.search(".blk$", file) and file != 'current.blk']
print(len(files))
files=files[:150]
BlockChain=pd.read_csv(files[0]).drop(0)
BlockChain=BlockChain.append([pd.read_csv(file,converters={"tx": literal_eval}) for file in files]).sort_values('block_index')

847


# Breaking down Blockchain into 4 datasets

In [ ]:
Tx=BlockChain['tx'].to_list()
Tx=pd.DataFrame([T for tx in Tx for T in tx])

In [ ]:
def transformer(S,Tx,I):
 for s in range(len(S)):
    S[s]['transaction-hash']=Tx['hash'].iloc[I]
 return S

In [ ]:
from numpy import nan
INs=Tx['inputs'].to_numpy()
INs=[transformer(S,Tx,I) for I,S in enumerate(INs)]
empty={'spent': nan,
 'script': nan,
 'spending_outpoints': nan,
 'tx_index': nan,
 'value': nan,
 'addr': nan,
 'n': nan,
 'type': nan}


OUTs=Tx['out'].to_numpy()
OUTs=[transformer(S,Tx,I) for I,S in enumerate(OUTs)]

    
INs=[p for x in INs for p in x]    
OUTs=[p for x in OUTs for p in x]

for _ in range(len(INs)):
    if(type(INs[_]['prev_out']) != dict):
     INs[_]['prev_out']=empty
    


INs=[{**S,**{'prev_out_'+key:value for key,value in S['prev_out'].items()}} for S in INs]

In [ ]:
IN=pd.DataFrame(INs).drop('prev_out',axis=1)
OUT=pd.DataFrame(OUTs)

# Cleaning and working with those datasets

In [ ]:
Important=['hash','block_index', 'time']
blockchain=BlockChain[Important]

Important=['hash','block_index', 'time']
tx=Tx[Important]
tx=tx.rename(columns={'hash':'index'})

Important=['transaction-hash', 'prev_out_value', 'prev_out_addr']
inp=IN[Important]
inp.columns=['transaction-id', 'value','addr']

Important=['transaction-hash', 'value','addr']
out=OUT[Important]
out=out.rename(columns={'transaction-hash':'transaction-id'})

In [ ]:
out['value']=out['value']*(-1)
inp_out=inp.append(out)

# Saving it for later access

In [ ]:
try:
 inp_out.append(pd.read_csv('inp_out.csv')).to_csv('inp_out.csv',index=False)
 tx.append(pd.read_csv('tx.csv')).to_csv('tx.csv',index=False)
 blockchain.append(pd.read_csv('blockchain.csv')).set_index('block_index').reset_index().to_csv('blockchain.csv',index=False)
except:
 blockchain.to_csv('blockchain.csv',index=False)
 inp_out.to_csv('inp_out.csv',index=False)
 tx.to_csv('tx.csv',index=False)


In [ ]:
!python Data\ Extraction.py
!python Data\ Extraction.py
!python Data\ Extraction.py
!python Data\ Extraction.py
!python Data\ Extraction.py
!python Data\ Extraction.py

847
